In [110]:
%reset -f
from pprint import pprint
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import rcParams
import seaborn as sns
sns.set()
rcParams['figure.figsize'] = 10,10
from sklearn.preprocessing import StandardScaler 
from functionsAll import lstmMultiSplit
import sklearn
import autosklearn.regression as regression

In [111]:
df = pd.read_csv("../Data/cleanned/zusammen.csv", parse_dates=["date"])
df['dayofyear'] = df['date'].dt.dayofyear
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['weekday'] = df['date'].dt.weekday
df['hour'] = df['date'].dt.hour
df['day'] = df['date'].dt.day
df['week'] = df['date'].dt.week
df.drop(columns=['date','rohwert'], inplace= True)
# Conclude with 11 features, 1 label.
df = df[[ 'wert', 'day', 'month', 'dayofyear', 'year', 'weekday', 'week', 'hour','temp', 'dwpt', 'rhum', 'prcp']]
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)

/home/cluster_home/yash/workspace/AutoMLRepo/Auto-ML-On-De-Pollution/autoMLVenv/lib64/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [112]:
y = df.pop('wert')
x = df 
trainX = x.query('index < 23000')
testX = x.query('index >= 23000')
trainY = y[:23000]
testY = y[23000:]
print(trainX.shape, testX.shape, trainY.shape, testY.shape)

(23000, 11) (736, 11) (23000,) (736,)


## Auto-Sklearn

In [113]:
autoML = regression.AutoSklearnRegressor(time_left_for_this_task=600,per_run_time_limit=60)
autoML.fit(trainX, trainY)

AutoSklearnRegressor(per_run_time_limit=60, time_left_for_this_task=600)

In [114]:
predict = autoML.predict(testX)

In [115]:
from sklearn.metrics import mean_absolute_error

In [116]:
mean_absolute_error(testY, predict)

5.630660290303438

In [69]:
prediction = np.repeat(predict, 12, axis=-1)
prediction.shape, predict.shape
# realPrediction = scaler.inverse_transform(prediction)
# realPrediction = realPrediction[:,0]
# yt = df.query('index >= 23000')
# yt = yt['wert']
# yt.reset_index(drop=True, inplace=True)

((8832,), (736,))

In [48]:
from pprint import pprint
pprint(autoML.show_models())

('[(1.000000, '
 "SimpleRegressionPipeline({'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': "
 "'no_encoding', "
 "'data_preprocessing:categorical_transformer:category_coalescence:__choice__': "
 "'minority_coalescer', "
 "'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', "
 "'data_preprocessing:numerical_transformer:rescaling:__choice__': "
 "'robust_scaler', 'feature_preprocessor:__choice__': 'no_preprocessing', "
 "'regressor:__choice__': 'k_nearest_neighbors', "
 "'data_preprocessing:categorical_transformer:category_coalescence:minority_coalescer:minimum_fraction': "
 '0.00397868572473159, '
 "'data_preprocessing:numerical_transformer:rescaling:robust_scaler:q_max': "
 '0.7840854273341944, '
 "'data_preprocessing:numerical_transformer:rescaling:robust_scaler:q_min': "
 "0.11376189722146844, 'regressor:k_nearest_neighbors:n_neighbors': 20, "
 "'regressor:k_nearest_neighbors:p': 2, "
 "'regressor:k_nearest_neighbors:weights': 'dista